<a href="https://colab.research.google.com/github/shere-khan/machine_learning/blob/master/TransformerEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset

import numpy as np
import matplotlib.pyplot as plt
# import torch_xla
# device = xm.xla_device()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_k, d_model, h):
    super().__init__()

    # Assume d_v = d_k
    # d_k: is size of the query, key, and value vectors
    # T: number of words in the sentence. denoted T_x in lectures
    self.d_k = d_k
    self.h   = h

    # Shape of single attention W^(Q) = d_model x d_k
    # Add multi-headed attention then W^(Q) = d_model x h*d_k

    # d_model: size of word embeddings. So d_model is
    # the size of whatever vector representation you
    # choose prior to input into the attention model. In the paper
    # d_model = 512
    self.key   = nn.Linear(d_model, h*d_k)
    self.query = nn.Linear(d_model, h*d_k)
    self.value = nn.Linear(d_model, h*d_k)

    #final linear layer
    self.fc = nn.Linear(d_k * h, d_model)

  def forward(self, q, k, v, mask=None):
    q = self.query(q) # NxTx(hd_k)
    k = self.key(k)   # NxTx(hd_k)
    v = self.value(v) # NxTx(hd_v)

    N = q.shape[0]
    T = q.shape[1]

    # change the shape to:
    # (N, T, h, d_k) -> (N, h, T, d_k)
    # in order for matrix multiply to work properly
    # view::returns the data transformed from
    # (N, T, h*d_k) -> (N, T, h, d_k)
    q = q.view(N, T, self.h, self.d_k).transpose(1, 2)
    k = k.view(N, T, self.h, self.d_k).transpose(1, 2)
    v = v.view(N, T, self.h, self.d_k).transpose(1, 2)

    # compute attention weights
    # (N, h, T, d_k) x (N, h, d_k, T) --> (N, h, T, T)
    attn_scores = q @ k.transpose(-2, -1) / math.sqrt(self.d_k)
    if mask is not None:
      # wherever the first argument is true, apply the
      # value of the second argument, which is -inf
      attn_scores = attn_scores.masked_fill(
          mask[:, None, None, :] == 0, float())
      # e^-inf = 0
      attn_weights = F.softmax(attn_scores, dim=-1)

    # compute attention-weighted values
    # (N, h, T, T) x (N, h, T, d_k) --> (N, h, T, d_k)
    A = attn_weights @ v

    # reshape it back before final linear layer
    A = A.transpose(1, 2) # (N, T, h, d_k)
    A = A.contiguous().view(N, T, self.d_k * self.h) # (N, T, h*d_k)

    # (N, T, h*d_k) x (h*d_k, d_model) -> (N, T, d_model)
    return self.fc(A)

In [ ]:
class TransformerBlock(nn.Module):
  def __init__(self, d_k, d_model, h, dropout_prob=.1):
    super().__init__()

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)
    self.mha = MultiHeadAttention(d_k, d_model, h)
    self.mlp = nn.Sequential(
      nn.Linear(d_model, d_model * 4),
      nn.GELU(),
      nn.Linear(d_model * 4, d_model),
      nn.Dropout(dropout_prob)
    )
    self.dropout = nn.Dropout(p=dropout_prob)

  def forward(self, x, mask=None):
    x = self.ln1(x + self.mha(x, x, x, mask))
    x = self.ln2(x + self.mlp(x))
    x = self.dropout(x)
    return x

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout_prob)

    position = torch.arange(max_len).unsqueeze(1) # counts across T
    exp_term = torch.arange(0, d_model, 2) # counts across feature dimension
    div_term = torch.exp(exp_term * (-math.log(10_000.0) / d_model))
    pe = torch.zeros(1, max_len, d_model)
    pe[0, :, 0::2] = torch.sin(position * div_term)
    pe[0, :, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, x):
    # x.shape: N x T x D
    x = x + self.pe[:, :x.size(1), :]
    return self.dropout(x)

In [ ]:
class Encoder(nn.Module):
  def __init__(self, vocab_size, max_len, d_k, d_model, h, n_layers, n_classes, dropout_prob):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size, d_model)
    self.pos_encoding = PositionalEncoding(d_model, max_len, dropout_prob)
    transformer_blocks = [
      TransformerBlock(
        d_k,
        d_model,
        h,
        dropout_prob) for _ in range(n_layers)
    ]
    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln = nn.LayerNorm(d_model)
    self.fc = nn.Linear(d_model, n_classes)

  def forward(self, x, mask=None):
    x = self.embedding(x)
    x = self.pos_encoding(x)
    for block in self.transformer_blocks:
      x = block(x, mask)

    # many-to-one (x has shape NxTxD)
    x = x[:, 0, :]
    x = self.ln(x)
    x = self.fc(x)
    return x

In [ ]:
# !pip install transformers datasets

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# distilbert does not use segment embeddings to
# distinguish bw multiple sentences in a single datapoint
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset('glue', 'sst2')
raw_datasets

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

data_collator

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column('label', 'labels')

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
  tokenized_datasets['train'],
  shuffle=True,
  batch_size=32,
  collate_fn=data_collator
)
valid_loader = DataLoader(
  tokenized_datasets['validation'],
  batch_size=32,
  collate_fn=data_collator
)

In [ ]:
# print out shape of data in batch
# input_ids.shape (32,47) (batch_size, T)
for batch in train_loader:
  for k, v in batch.items():
    print(f'{k}, v.shape: {v.shape}')
  break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


labels, v.shape: torch.Size([32])
input_ids, v.shape: torch.Size([32, 50])
attention_mask, v.shape: torch.Size([32, 50])


In [ ]:
set(tokenized_datasets['train']['labels'])

{0, 1}

In [ ]:
tokenizer.vocab_size

28996

In [ ]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [ ]:
model = Encoder(
  vocab_size=tokenizer.vocab_size,
  max_len=tokenizer.max_model_input_sizes[checkpoint],
  d_k=16,
  d_model=64,
  h=4,
  n_layers=2,
  n_classes=2,
  dropout_prob=.1)

model.to(device)

Encoder(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, 

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from datetime import datetime

In [ ]:
def train(model, criterion, optimizer, train_loader, valid_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = 0
    n_train = 0
    for batch in train_loader:
      # move data to GPU
      batch = {k: v.to(device) for k, v in batch.items()}

      # zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs, batch['labels'])

      # backward to compute gradients
      loss.backward()
      # one step of gradient descent
      optimizer.step()

      # multiply by batch size to turn average back into sum
      train_loss += loss.item() * batch['input_ids'].size(0)
      # then total up the sums across batches
      n_train += batch['input_ids'].size(0)
    # get average train loss
    train_loss = train_loss / n_train

    # Model evaluation
    model.eval()
    test_loss = 0
    n_test = 0
    for batch in valid_loader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs, batch['labels'])
      test_loss += loss.item() * batch['input_ids'].size(0)
      n_test += batch['input_ids'].size(0)
    test_loss = test_loss / n_test

    # Save losses
    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0
    print(f"""Epoch {it + 1}/{epochs}, Train Loss: {train_loss:.4f},
    Test Loss: {test_loss:.4f}, Duration: {dt}""")
    
  return train_losses, test_losses

In [ ]:
train_losses, test_losses = train(
  model, criterion, optimizer, train_loader, valid_loader, epochs=4)

Epoch 1/4, Train Loss: 0.2445,
    Test Loss: 0.4949, Duration: 0:00:26.942797
Epoch 2/4, Train Loss: 0.2223,
    Test Loss: 0.4885, Duration: 0:00:24.264499
Epoch 3/4, Train Loss: 0.2049,
    Test Loss: 0.5168, Duration: 0:00:24.171910
Epoch 4/4, Train Loss: 0.1888,
    Test Loss: 0.5565, Duration: 0:00:24.688117


In [ ]:
# train accuracy

model.eval()
n_correct = 0.
n_total = 0.
for batch in train_loader:
  # move to gpu
  batch = {k: v.to(device) for k, v in batch.items()}

  # Forward pass
  outputs = model(batch['input_ids'], batch['attention_mask'])

  # Get prediction
  # torch.max returns both max and argmax
  _, predictions = torch.max(outputs, 1)

  # update counts
  n_correct += (predictions == batch['labels']).sum().item()
  n_total += batch['labels'].shape[0]

train_acc = n_correct / n_total

In [ ]:
# test accuracy

n_correct = 0.
n_total = 0.
for batch in valid_loader:
  # move to gpu
  batch = {k: v.to(device) for k, v in batch.items()}

  # Forward pass
  outputs = model(batch['input_ids'], batch['attention_mask'])

  # Get prediction
  # torch.max returns both max and argmax
  _, predictions = torch.max(outputs, 1)

  # update counts
  n_correct += (predictions == batch['labels']).sum().item()
  n_total += batch['labels'].shape[0]

test_acc = n_correct / n_total
print(f'Train acc: {train_acc:.4f}, Test acc: {test_acc:.4f}')

Train acc: 0.9527, Test acc: 0.7993
